This file contains the code for ND, DMD, TVDMD including

- regression:  double reg, first for selecting feaures and second for AUC and prediction

In [50]:
import sys
sys.path.append("/home/huaqingj/MyResearch/TVDN-AD/")

In [51]:
import importlib
from pyTVDN import TVDNDetect

import pyTVDN.utils
importlib.reload(pyTVDN.utils)
from pyTVDN.utils import load_pkl, save_pkl

from sklearn.cluster import KMeans
from pathlib import Path
from scipy.io import loadmat
import numpy as np
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
import os
from scipy import signal
from scipy.stats import fisher_exact, norm
import pickle
import seaborn as sns
#from tqdm.autonotebook import tqdm
from tqdm import tqdm
import numbers
import pandas as pd
%matplotlib inline

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score, roc_curve

import warnings
warnings.filterwarnings('ignore')

### Load data

In [53]:
os.chdir("/home/huaqingj/MyResearch/TVDN-AD")
resDir = Path("./results")
dataDir = Path("./data")

In [54]:
def wUFn(Amat):
    eigVals, eigVecs = np.linalg.eig(Amat)
    r = np.sum(np.cumsum(np.abs(eigVals))/np.sum(np.abs(eigVals)) <= 0.8) + 1
    if (eigVals[r-1].imag + eigVals[r].imag ) == 0:
        r = r + 1
    wU = np.abs(np.matmul(eigVecs[:, :r], eigVals[:r]))
    return wU, r, eigVals[:r]

In [55]:
def minmax(x):
    num = x - np.min(x)
    den = np.max(x) - np.min(x)
    return num/den

def U2BrainVec(wU):
    emVec = np.zeros_like(DKtmplate, dtype=np.float64)
    for idx in range(1, 69):
        emVec[DKtmplate==idx] = wU[idx-1]
    return emVec

def reorderU(wU):
    wUreorder = np.zeros_like(wU, dtype=np.float64)
    wUreorder[0::2] = wU[:34]
    wUreorder[1::2] = wU[34:]
    return wUreorder

In [56]:
def TuningCFn(inpX, inpY, Cs=[0.1, 0.2, 0.4, 0.8, 1, 1.6, 3.2, 6.4, 12.8, 25.6], penalty="l2"):
    aucCs = []
    for C in Cs:
        eProbs = []
        loo = LeaveOneOut()
        for trIdxs, testIdxs in loo.split(inpX):
            clf = LogisticRegression(penalty=penalty, random_state=0, C=C)
            clf.fit(inpX[trIdxs, :], inpY[trIdxs])
            eProbs.append(clf.predict_proba(inpX[testIdxs, :]))
        eProbs = np.array(eProbs).squeeze()
        auc = roc_auc_score(inpY, eProbs[:, 1])
        fpr, tpr, thresholds = roc_curve(inpY, eProbs[:, 1], pos_label=1)
        aucCs.append(auc)
            
    optC = Cs[np.argmax(aucCs)]
    res = edict()
    res["optC"] = optC
    res["Cs"] = Cs
    res["aucCs"] = aucCs
    return res

In [57]:
def rSelFn(Amat):
    eigVals, eigVecs = np.linalg.eig(Amat)
    rSel = np.where(np.cumsum(np.abs(eigVals))/np.sum(np.abs(eigVals)) >0.8)[0][0] + 1
    # if breaking conjugate eigval pair, add r with 1
    if (eigVals[rSel-1].imag + eigVals[rSel].imag) == 0:
        rSel = rSel + 1
    wU = np.abs(np.matmul(eigVecs[:, :rSel], eigVals[:rSel]))
    return rSel, wU, eigVals[:rSel]

### Parameters

In [58]:
Cs = [0.001, 0.01, 0.05, 0.1, 0.2, 0.4, 0.8, 1, 1.6, 3.2, 6.4, 12.8, 25.6]
penalty = "l2"
freq = 120

### DMD

In [59]:
filName = f"Ctrl_data_NWM_DMD.pkl"
with open(resDir/filName, "rb") as f:
    CtrlDMD = pickle.load(f)
filName = f"AD_data_NWM_DMD.pkl"
with open(resDir/filName, "rb") as f:
    ADDMD = pickle.load(f)

In [60]:
fsAll = []
for v in ADDMD:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsAD = fsAll

In [61]:
fsAll = []
for v in CtrlDMD:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsCtrl = fsAll

In [62]:
fsName = ["r", "tQ", "tL", "wU"]
#fsName = ["r", "tQ", "tC", "tL", "wU"]
#fsName = ["r", "absImagEigValMaxMean", "tQ", "tC", "tL", "wU"]

Ys = np.concatenate([np.ones(len(fsAD)), np.zeros(len(fsCtrl))])
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0)
stdXs = (Xs - Xs.mean(axis=0))/Xs.std(axis=0)
    

#### First fit

In [63]:
## fit
gOptC1 = TuningCFn(stdXs, Ys, Cs, penalty)["optC"]
print(gOptC1)

# fit the first reg
clfFinal = LogisticRegression(penalty=penalty, random_state=0, C=gOptC1)
clfFinal.fit(stdXs, Ys)
coefsFinal = clfFinal.coef_.reshape(-1)

0.01


In [64]:
# boostrap for pval
np.random.seed(0)
repTime = 10000
parassBoot = []
for i in tqdm(range(repTime)):
    bootIdx = np.random.choice(len(Ys), len(Ys))
    YsBoot = Ys[bootIdx]
    stdXsBoot = stdXs[bootIdx]
    clf = LogisticRegression(penalty=penalty, random_state=0, C=gOptC1)
    clf.fit(stdXsBoot, YsBoot)
    parasBoot = clf.coef_.reshape(-1)
    parassBoot.append(parasBoot)

100%|██████████| 10000/10000 [00:07<00:00, 1265.15it/s]


In [65]:
parassBoot = np.array(parassBoot)
#lows, ups = np.quantile(parassBoot[:, :], [0.025, 0.975], axis=0)
lows, ups = coefsFinal-parassBoot.std(axis=0)*1.96, coefsFinal+parassBoot.std(axis=0)*1.96
kpidxBoot = np.bitwise_or(lows >0,  ups < 0)
select_fs_idx = kpidxBoot
select_fs_idx.sum()
np.where(select_fs_idx)[0]

array([ 0,  9, 16, 28, 40, 42, 45, 51, 60, 62, 65])

In [66]:
# Pvalue
testStatsBoot = coefsFinal/parassBoot.std(axis=0)
#testStatsBoot = parassBoot.mean(axis=0)/parassBoot.std(axis=0)
normRV = norm()
# to be consistent, I think we should use two-sided pvalue
bootPvs = 2*(1-normRV.cdf(np.abs(testStatsBoot)))

num_other_fs = len(fsName) - 1
resDic = {
   "Feature" : np.array(fsName)[:num_other_fs],
    "Parameters": coefsFinal[:num_other_fs],
    "Lower": lows[:num_other_fs],
    "Upper": ups[:num_other_fs],
    "Pvalues": bootPvs[:num_other_fs]
}
#pd.set_option("display.precision", 3)
pd.set_option('display.float_format',lambda x : '%.3e' % x)
resDF = pd.DataFrame(resDic)
print(resDF)
print(resDF.to_latex(float_format="{:0.3f}".format))

  Feature  Parameters      Lower     Upper   Pvalues
0       r   1.307e-01  7.213e-02 1.892e-01 1.214e-05
1      tQ  -6.277e-02 -1.316e-01 6.084e-03 7.397e-02
2      tL  -2.239e-02 -9.658e-02 5.179e-02 5.541e-01
\begin{tabular}{llrrrr}
\toprule
{} & Feature &  Parameters &  Lower &  Upper &  Pvalues \\
\midrule
0 &       r &       0.131 &  0.072 &  0.189 &    0.000 \\
1 &      tQ &      -0.063 & -0.132 &  0.006 &    0.074 \\
2 &      tL &      -0.022 & -0.097 &  0.052 &    0.554 \\
\bottomrule
\end{tabular}



#### Second fit

In [67]:
# select fs
stdXs_selected = stdXs[:, select_fs_idx]
stdXs_selected.shape

(176, 11)

In [68]:
gOptC2 = TuningCFn(stdXs_selected, Ys, Cs, penalty=penalty)["optC"]
print(gOptC2)

0.05


In [69]:
eProbs = []
loo = LeaveOneOut()
parass = []
with tqdm(total=len(Ys)) as t:
    for trIdxs, testIdxs in loo.split(stdXs_selected):
        curOptC = TuningCFn(stdXs_selected[trIdxs, :], Ys[trIdxs], Cs, penalty)["optC"]
        clf = LogisticRegression(penalty=penalty, random_state=0, C=curOptC)
        clf.fit(stdXs_selected[trIdxs, :], Ys[trIdxs])
        paras = np.concatenate([clf.intercept_, clf.coef_.reshape(-1)])
        parass.append(paras)
        eProbs.append(clf.predict_proba(stdXs_selected[testIdxs, :]))
        t.update()
eProbs = np.array(eProbs).squeeze()
auc = roc_auc_score(Ys, eProbs[:, 1])
fpr, tpr, thresholds = roc_curve(Ys, eProbs[:, 1], pos_label=1)
parass = np.array(parass)
sfName = "_".join(fsName)
with open(resDir/f"DMD_ROCcurve_reg2_{sfName}.pkl", "wb") as f:
    resROC = {"fpr":fpr, "tpr":tpr, "AUC":auc}
    pickle.dump(resROC, f)


np.random.seed(0)
nobs = stdXs_selected.shape[0]
Aucss = []
for j in tqdm(range(10000)):
    testIdx = np.random.choice(nobs, int(nobs/5), False)
    trainIdx = np.delete(np.arange(nobs), testIdx)
    clf = LogisticRegression(penalty=penalty, random_state=0, C=gOptC2)
    clf.fit(stdXs_selected[trainIdx], Ys[trainIdx])
    curEprobs = clf.predict_proba(stdXs_selected[testIdx, :])
    curAuc = roc_auc_score(Ys[testIdx], curEprobs[:, 1])
    Aucss.append(curAuc)
mAUC = np.mean(Aucss)
stdAUC = np.std(Aucss)
print(f"Features are {fsName}.")
print(f"The AUC under optimal C is {auc:.3f}.")
print(f"The mean of AUC under 1000 repetitions is {mAUC:.3f} and the standard deviation is {stdAUC:.3f}.")
print("-"*100)

100%|██████████| 10000/10000 [00:10<00:00, 950.26it/s]

Features are ['r', 'tQ', 'tL', 'wU'].
The AUC under optimal C is 0.756.
The mean of AUC under 1000 repetitions is 0.762 and the standard deviation is 0.074.
----------------------------------------------------------------------------------------------------


### ND(TV)

In [70]:
filName = f"Ctrl_data_NWM_TV.pkl"
with open(resDir/filName, "rb") as f:
    CtrlTV = pickle.load(f)
filName = f"AD_data_NWM_TV.pkl"
with open(resDir/filName, "rb") as f:
    ADTV = pickle.load(f)

In [71]:
fsAll = []
for v in ADTV:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsAD = fsAll

In [72]:
fsAll = []
for v in CtrlTV:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsCtrl = fsAll

In [73]:
fsName = ["r", "tQ", "tL", "wU"]
#fsName = ["r", "tQ", "tC", "tL", "wU"]
#fsName = ["r", "absImagEigValMaxMean", "tQ", "tC", "tL", "wU"]
Ys = np.concatenate([np.ones(len(fsAD)), np.zeros(len(fsCtrl))])
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0)
stdXs = (Xs - Xs.mean(axis=0))/Xs.std(axis=0)

#### First fit

In [74]:
## fit
gOptC1 = TuningCFn(stdXs, Ys, Cs, penalty)["optC"]
print(gOptC1)

# fit the first reg
clfFinal = LogisticRegression(penalty=penalty, random_state=0, C=gOptC1)
clfFinal.fit(stdXs, Ys)
coefsFinal = clfFinal.coef_.reshape(-1)

0.01


In [75]:
# boostrap for pval
np.random.seed(0)
repTime = 10000
parassBoot = []
for i in tqdm(range(repTime)):
    bootIdx = np.random.choice(len(Ys), len(Ys))
    YsBoot = Ys[bootIdx]
    stdXsBoot = stdXs[bootIdx]
    clf = LogisticRegression(penalty=penalty, random_state=0, C=gOptC1)
    clf.fit(stdXsBoot, YsBoot)
    parasBoot = clf.coef_.reshape(-1)
    parassBoot.append(parasBoot)

100%|██████████| 10000/10000 [00:08<00:00, 1232.95it/s]


In [76]:
parassBoot = np.array(parassBoot)
#lows, ups = np.quantile(parassBoot[:, :], [0.025, 0.975], axis=0)
lows, ups = coefsFinal-parassBoot.std(axis=0)*1.96, coefsFinal+parassBoot.std(axis=0)*1.96
kpidxBoot = np.bitwise_or(lows >0,  ups < 0)
select_fs_idx = kpidxBoot
select_fs_idx.sum()
np.where(select_fs_idx)[0]

array([ 0,  4, 14, 20, 24, 34, 41, 43, 48, 52, 59, 62, 64, 69])

In [77]:
# Pvalue
testStatsBoot = coefsFinal/parassBoot.std(axis=0)
#testStatsBoot = parassBoot.mean(axis=0)/parassBoot.std(axis=0)
normRV = norm()
# to be consistent, I think we should use two-sided pvalue
bootPvs = 2*(1-normRV.cdf(np.abs(testStatsBoot)))

num_other_fs = len(fsName) - 1
resDic = {
   "Feature" : np.array(fsName)[:num_other_fs],
    "Parameters": coefsFinal[:num_other_fs],
    "Lower": lows[:num_other_fs],
    "Upper": ups[:num_other_fs],
    "Pvalues": bootPvs[:num_other_fs]
}
#pd.set_option("display.precision", 3)
pd.set_option('display.float_format',lambda x : '%.3e' % x)
resDF = pd.DataFrame(resDic)
print(resDF)
print(resDF.to_latex(float_format="{:0.3f}".format))

  Feature  Parameters      Lower     Upper   Pvalues
0       r   1.042e-01  4.169e-02 1.667e-01 1.085e-03
1      tQ   4.010e-02 -3.457e-02 1.148e-01 2.925e-01
2      tL  -1.106e-02 -8.687e-02 6.474e-02 7.749e-01
\begin{tabular}{llrrrr}
\toprule
{} & Feature &  Parameters &  Lower &  Upper &  Pvalues \\
\midrule
0 &       r &       0.104 &  0.042 &  0.167 &    0.001 \\
1 &      tQ &       0.040 & -0.035 &  0.115 &    0.293 \\
2 &      tL &      -0.011 & -0.087 &  0.065 &    0.775 \\
\bottomrule
\end{tabular}



#### Second fit

In [78]:
# select fs
stdXs_selected = stdXs[:, select_fs_idx]
stdXs_selected.shape

(176, 14)

In [79]:
gOptC2 = TuningCFn(stdXs_selected, Ys, Cs, penalty=penalty)["optC"]
print(gOptC2)

0.1


In [80]:
eProbs = []
loo = LeaveOneOut()
parass = []
with tqdm(total=len(Ys)) as t:
    for trIdxs, testIdxs in loo.split(stdXs_selected):
        curOptC = TuningCFn(stdXs_selected[trIdxs, :], Ys[trIdxs], Cs, penalty)["optC"]
        clf = LogisticRegression(penalty=penalty, random_state=0, C=curOptC)
        clf.fit(stdXs_selected[trIdxs, :], Ys[trIdxs])
        paras = np.concatenate([clf.intercept_, clf.coef_.reshape(-1)])
        parass.append(paras)
        eProbs.append(clf.predict_proba(stdXs_selected[testIdxs, :]))
        t.update()
eProbs = np.array(eProbs).squeeze()
auc = roc_auc_score(Ys, eProbs[:, 1])
fpr, tpr, thresholds = roc_curve(Ys, eProbs[:, 1], pos_label=1)
parass = np.array(parass)
sfName = "_".join(fsName)
with open(resDir/f"ND_ROCcurve_reg2_{sfName}.pkl", "wb") as f:
    resROC = {"fpr":fpr, "tpr":tpr, "AUC":auc}
    pickle.dump(resROC, f)


np.random.seed(0)
nobs = stdXs_selected.shape[0]
Aucss = []
for j in tqdm(range(10000)):
    testIdx = np.random.choice(nobs, int(nobs/5), False)
    trainIdx = np.delete(np.arange(nobs), testIdx)
    clf = LogisticRegression(penalty=penalty, random_state=0, C=gOptC2)
    clf.fit(stdXs_selected[trainIdx], Ys[trainIdx])
    curEprobs = clf.predict_proba(stdXs_selected[testIdx, :])
    curAuc = roc_auc_score(Ys[testIdx], curEprobs[:, 1])
    Aucss.append(curAuc)
mAUC = np.mean(Aucss)
stdAUC = np.std(Aucss)
print(f"Features are {fsName}.")
print(f"The AUC under optimal C is {auc:.3f}.")
print(f"The mean of AUC under 1000 repetitions is {mAUC:.3f} and the standard deviation is {stdAUC:.3f}.")
print("-"*100)

100%|██████████| 10000/10000 [00:11<00:00, 908.12it/s]

Features are ['r', 'tQ', 'tL', 'wU'].
The AUC under optimal C is 0.754.
The mean of AUC under 1000 repetitions is 0.765 and the standard deviation is 0.074.
----------------------------------------------------------------------------------------------------


### Dynamic DMD

In [125]:
filName = f"AD_data_dynDMD_withgraph.pkl"
with open(resDir/filName, "rb") as f:
    fsAD = pickle.load(f)
    
filName = f"Ctrl_data_dynDMD_withgraph.pkl"
with open(resDir/filName, "rb") as f:
    fsCtrl = pickle.load(f)
    
# for abs imag max eig
filName = f"AD_data_dynDMD.pkl"
with open(resDir/filName, "rb") as f:
    fsAD_add = pickle.load(f)
    
filName = f"Ctrl_data_dynDMD.pkl"
with open(resDir/filName, "rb") as f:
    fsCtrl_add = pickle.load(f)

In [126]:
map_dict = ["tmC", "tmL", "tQ", "C", "L", "Q"]

In [127]:
fs_TVDMD = fsAD
fs_TVDMD_add = fsAD_add

fsAll = []
for ix in range(len(fs_TVDMD)):
    raw_fs = edict(fs_TVDMD[ix])
    add_fs = edict(fs_TVDMD_add[ix])
    fs = edict()
    
    fs.r = raw_fs.r
    fs.ncpts = raw_fs.ncpts
    fs.absEigValMean = raw_fs.absEigValMean
    fs.wUmeanAll = raw_fs.wUmeanAll
    fs.absImagEigValMaxMean = add_fs.absImagEigValMaxMean
    
    fs.var_C = np.var([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.var_L = np.var([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.var_Q = np.var([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    fs.mean_C = np.mean([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.mean_L = np.mean([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.mean_Q = np.mean([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    dwell = np.diff(np.concatenate([[0], raw_fs.ecpts, [7200]])/freq)
    max_dwell_idx = np.argsort(-dwell)[0]
    fs.top1_C = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmC")]
    fs.top1_L = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmL")]
    fs.top1_Q = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tQ")]
    fs.max_dwell = np.max(dwell)
    
    
    fsAll.append(fs)
    
fsAD_trans = fsAll

In [128]:
fs_TVDMD = fsCtrl
fs_TVDMD_add = fsCtrl_add

fsAll = []
for ix in range(len(fs_TVDMD)):
    raw_fs = edict(fs_TVDMD[ix])
    add_fs = edict(fs_TVDMD_add[ix])
    fs = edict()
    
    fs.r = raw_fs.r
    fs.ncpts = raw_fs.ncpts
    fs.absEigValMean = raw_fs.absEigValMean
    fs.wUmeanAll = raw_fs.wUmeanAll
    fs.absImagEigValMaxMean = add_fs.absImagEigValMaxMean
    
    fs.var_C = np.var([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.var_L = np.var([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.var_Q = np.var([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    fs.mean_C = np.mean([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.mean_L = np.mean([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.mean_Q = np.mean([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    dwell = np.diff(np.concatenate([[0], raw_fs.ecpts, [7200]])/freq)
    max_dwell_idx = np.argsort(-dwell)[0]
    fs.top1_C = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmC")]
    fs.top1_L = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmL")]
    fs.top1_Q = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tQ")]
    fs.max_dwell = np.max(dwell)
    
    
    fsAll.append(fs)
    
fsCtrl_trans = fsAll

In [129]:
penalty = "l2"
Ys = np.concatenate([np.ones(len(fsAD_trans)), np.zeros(len(fsCtrl_trans))])
fsName = ["r", "ncpts",# "absImagEigValMaxMean", 
          "mean_Q", "mean_L", 
          "var_Q",  "var_L",  
          "wUmeanAll" ]
#fsName = ["r", "ncpts",# "absImagEigValMaxMean", 
#          "top1_Q", "top1_C", "top1_L", 
#          "var_Q",  "var_C",  "var_L",  
#          "wUmeanAll" ]
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD_trans[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD_trans]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl_trans]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD_trans])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl_trans])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0)
stdXs = (Xs - Xs.mean(axis=0))/Xs.std(axis=0)

# name prefix to save the results
preFix = "_".join(fsName) + f"_{penalty}"
print(preFix)

r_ncpts_mean_Q_mean_L_var_Q_var_L_wUmeanAll_l2


In [130]:
fstCtrl = [ix.ncpts for ix in fsCtrl_trans]
fstAD = [ix.ncpts for ix in fsAD_trans]
np.mean(fstAD), np.mean(fstCtrl)

(67.80681818181819, 52.875)

In [131]:
import scipy
scipy.stats.ttest_ind(fstAD, fstCtrl)

Ttest_indResult(statistic=3.3319505717639197, pvalue=0.0010533389057732372)

In [132]:
fstCtrl = [ix.r for ix in fsCtrl_trans]
fstAD = [ix.r for ix in fsAD_trans]
print(np.mean(fstAD), np.mean(fstCtrl))
scipy.stats.ttest_ind(fstAD, fstCtrl)

4.829545454545454 4.306818181818182


Ttest_indResult(statistic=2.8302418458676666, pvalue=0.00519874795155934)

In [133]:
fstCtrl = [ix.max_dwell for ix in fsCtrl_trans]
fstAD = [ix.max_dwell for ix in fsAD_trans]

print(np.mean(fstAD), np.mean(fstCtrl))

print(scipy.stats.ttest_ind(fstAD, fstCtrl))

7.761363636363637 8.908901515151515
Ttest_indResult(statistic=-2.4028302492610254, pvalue=0.017320350636294994)


#### Fisrt fit

In [141]:
# The opTuningC will be used throughout
cur_Cs = [1e-20, 0.0005, 0.001, 0.01, 0.05, 0.1, 0.2, 0.4, 0.8, 1, 1.6, 3.2, 6.4, 12.8, 25.6]
gOptC1 = TuningCFn(stdXs, Ys, cur_Cs, penalty=penalty)["optC"]
print(gOptC1)

# fit the first reg
clfFinal = LogisticRegression(penalty=penalty, random_state=0, C=gOptC1)
clfFinal.fit(stdXs, Ys)
coefsFinal = clfFinal.coef_.reshape(-1)

0.0005


In [142]:
# boostrap for pvals
np.random.seed(0)
repTime = 10000
parassBoot = []
for i in tqdm(range(repTime)):
    bootIdx = np.random.choice(len(Ys), len(Ys))
    YsBoot = Ys[bootIdx]
    stdXsBoot = stdXs[bootIdx]
    clf = LogisticRegression(penalty=penalty, random_state=0, C=gOptC1)
    clf.fit(stdXsBoot, YsBoot)
    parasBoot = clf.coef_.reshape(-1)
    parassBoot.append(parasBoot)
    
    
parassBoot = np.array(parassBoot)
#lows, ups = np.quantile(parassBoot[:, :], [0.025, 0.975], axis=0)
lows, ups = coefsFinal-parassBoot.std(axis=0)*1.96, coefsFinal+parassBoot.std(axis=0)*1.96
kpidxBoot = np.bitwise_or(lows >0,  ups < 0)
kpidxIdBoot = np.where(kpidxBoot)[0]


select_fs_idx = kpidxBoot

100%|██████████| 10000/10000 [00:06<00:00, 1504.35it/s]


In [143]:
# Pvalue
testStatsBoot = coefsFinal/parassBoot.std(axis=0)
#testStatsBoot = parassBoot.mean(axis=0)/parassBoot.std(axis=0)
normRV = norm()
# to be consistent, I think we should use two-sided pvalue
bootPvs = 2*(1-normRV.cdf(np.abs(testStatsBoot)))

num_other_fs = len(fsName) - 1
resDic = {
   "Feature" : np.array(fsName)[:num_other_fs],
    "Parameters": coefsFinal[:num_other_fs],
    "Lower": lows[:num_other_fs],
    "Upper": ups[:num_other_fs],
    "Pvalues": bootPvs[:num_other_fs]
}
#pd.set_option("display.precision", 3)
pd.set_option('display.float_format',lambda x : '%.3e' % x)
resDF = pd.DataFrame(resDic)
print(resDF)
print(resDF.to_latex(float_format="{:0.3f}".format))

  Feature  Parameters      Lower     Upper   Pvalues
0       r   8.197e-03  2.486e-03 1.391e-02 4.906e-03
1   ncpts   9.826e-03  4.177e-03 1.547e-02 6.505e-04
2  mean_Q  -7.502e-05 -5.917e-03 5.767e-03 9.799e-01
3  mean_L  -4.551e-03 -1.021e-02 1.108e-03 1.150e-01
4   var_Q   8.256e-03  2.584e-03 1.393e-02 4.329e-03
5   var_L   9.517e-03  3.869e-03 1.516e-02 9.575e-04
\begin{tabular}{llrrrr}
\toprule
{} & Feature &  Parameters &  Lower &  Upper &  Pvalues \\
\midrule
0 &       r &       0.008 &  0.002 &  0.014 &    0.005 \\
1 &   ncpts &       0.010 &  0.004 &  0.015 &    0.001 \\
2 &  mean\_Q &      -0.000 & -0.006 &  0.006 &    0.980 \\
3 &  mean\_L &      -0.005 & -0.010 &  0.001 &    0.115 \\
4 &   var\_Q &       0.008 &  0.003 &  0.014 &    0.004 \\
5 &   var\_L &       0.010 &  0.004 &  0.015 &    0.001 \\
\bottomrule
\end{tabular}



#### Second fit

In [144]:
# select fs
stdXs_selected = stdXs[:, select_fs_idx]
print(kpidxIdBoot)

[ 0  1  4  5  9 19 27 38 45 46 48 53 54 57 61 62 64]


In [145]:
gOptC2 = TuningCFn(stdXs_selected, Ys, cur_Cs, penalty=penalty)["optC"]
print(gOptC2)

0.0005


In [146]:
eProbs = []
loo = LeaveOneOut()
parass = []
curOptCs = []
with tqdm(total=len(Ys)) as t:
    for trIdxs, testIdxs in loo.split(stdXs_selected):
        curStdXs, curYs = stdXs_selected[trIdxs, :], Ys[trIdxs]
        curOptC = TuningCFn(curStdXs, curYs, cur_Cs, penalty=penalty)["optC"]
        curOptCs.append(curOptC)
        clf = LogisticRegression(penalty=penalty, random_state=0, C=curOptC)
        clf.fit(curStdXs, curYs)
        paras = clf.coef_.reshape(-1)
        parass.append(paras)
        eProbs.append(clf.predict_proba(stdXs_selected[testIdxs, :]))
        
        t.update()
eProbs = np.array(eProbs).squeeze()
gAuc = roc_auc_score(Ys, eProbs[:, 1])
gFpr, gTpr, thresholds = roc_curve(Ys, eProbs[:, 1], pos_label=1)
parass = np.array(parass)
print(gAuc)

with open(resDir/f"TVDMD_ROCcurve_reg2_{preFix}.pkl", "wb") as f:
    resROC = {"fpr":gFpr, "tpr":gTpr, "AUC":gAuc, "curOptCs":curOptCs}
    pickle.dump(resROC, f)

100%|██████████| 176/176 [04:33<00:00,  1.55s/it]

0.7697572314049586


In [147]:
np.random.seed(0)
nobs = stdXs_selected.shape[0]
Aucss = []
for j in tqdm(range(10000)):
    testIdx = np.random.choice(nobs, int(nobs/5), False)
    trainIdx = np.delete(np.arange(nobs), testIdx)
    clf = LogisticRegression(penalty=penalty, random_state=0, C=gOptC2)
    clf.fit(stdXs_selected[trainIdx], Ys[trainIdx])
    curEprobs = clf.predict_proba(stdXs_selected[testIdx, :])
    curAuc = roc_auc_score(Ys[testIdx], curEprobs[:, 1])
    Aucss.append(curAuc)
mAUC = np.mean(Aucss)
stdAUC = np.std(Aucss)
print(f"The mean of AUC under 1000 repetitions is {mAUC:.3f} and the standard deviation is {stdAUC:.3f}, "
      f"the 95% CI is ({np.quantile(Aucss, 0.025):.3f}, {np.quantile(Aucss, 0.975):.3f}).")

100%|██████████| 10000/10000 [00:10<00:00, 965.29it/s]

The mean of AUC under 1000 repetitions is 0.774 and the standard deviation is 0.071, the 95% CI is (0.627, 0.903).
